In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Парсинг данных

###Панорама:

То, что парсила Аня

In [ ]:
import requests, json, concurrent, pandas as pd
from bs4 import BeautifulSoup

site_categories = []

main_page_content = requests.get(f'https://panorama.pub').content
bs_instance = BeautifulSoup(main_page_content, 'html.parser')
useless_categories_links = ['/', '/books', '/donate', '/about']
for nav_link in bs_instance.find_all('a', class_='nav-link'):
  if nav_link['href'] in useless_categories_links:
    continue
  site_categories.append({
      'link': nav_link['href'],
      'name': nav_link.text.replace('\n', '').strip()
  })

parsed_news = {site_category['name']: [] for site_category in site_categories}
def get_pages(category, pages_amount, page_num):
  print(f'Скачиваю страницу номер {page_num} из {pages_amount}')
  page_content = requests.get(f'https://panorama.pub{category["link"]}?page={page_num}').content

  for news_link in BeautifulSoup(page_content, 'html.parser').find_all('a', class_='entry big'):
    news_content = requests.get(f'https://panorama.pub{news_link["href"]}').content
    bs_instance = BeautifulSoup(news_content, 'html.parser')

    news_obj = {
        'title': bs_instance.find('h1').text.replace('\n', '').strip(),
        'text': ' '.join([tag.text for tag in bs_instance.find_all('p')]) 
    }
    parsed_news[category['name']].append(news_obj)
    print(news_obj['title'])


for category in site_categories:
  print('#' * 20)
  print(f'Скачиваю категорию: {category["name"]}')
  category_content = requests.get(f'https://panorama.pub{category["link"]}').content

  bs_instance = BeautifulSoup(category_content, 'html.parser')
  pages_amount = int(bs_instance.find('ul', class_='pagination').find_all('li')[-2].select('a')[0].text)
  with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
    res = [executor.submit(get_pages, category, pages_amount, page_num) for page_num in range(1, pages_amount + 1)]
    concurrent.futures.wait(res)
      
    # with open('/content/dump_articles.json', 'w') as f:
    #   f.write(json.dumps(parsed_news))
  print('#' * 20)

data_list = []
for category in site_categories:
  for news in parsed_news[category['name']]:
    data_list.append({
      'Заголовок': news['title'],
      'Текст': news['text'],
       'Категория': category['name']
    })
pd.DataFrame(data_list)

In [ ]:
pd.DataFrame(data_list).to_csv('/content/panorama_all.csv', encoding='utf-8', index=False)

Соединяем все в одно

In [ ]:
anna = pd.read_csv('/content/panorama_all.csv')

In [ ]:
gleb = pd.read_csv('economics.csv')
for name in ['society.csv', 'articles.csv', 'politics.csv', 'books.csv', 'science.csv']:
    buff = pd.read_csv(name)
    gleb = pd.concat([gleb, buff])

In [ ]:
gleb = gleb.loc[:, ['Category', 'Header', 'Text']]

In [ ]:
def tranform(x):
    if x in ['«Экономика»', '«Общество»', '«Статьи»', '«Политика»', '«Книги»', '«Наука»']:
        return str(x)[1:-1]

In [ ]:
gleb.Category = list(map(tranform  , gleb.Category.tolist()))

In [ ]:
missing_val_count_by_column = (gleb.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [ ]:
gleb.dropna(inplace= True)

In [ ]:
anna.rename(columns={'Заголовок': 'Header', 'Текст': 'Text', 'Категория': 'Category'}, inplace= True)
anna.head()

,Header,Text,Category
0,В Берлине прошли народные гулянья в честь годо...,Более 4 тысяч берлинцев приняли участие в наро...,Политика
1,Жители Бендер требуют переименования города,Более 20 тысяч жителей молдавского города Бенд...,Политика
2,Меркель и Лукашенко договорились разделить Пол...,Врио канцлера ФРГ Ангела Меркель и Александр Л...,Политика
3,Либерализация в КНДР: пограничники будут дават...,"Лидер КНДР Ким Чен Ын объявил о том, что в рам...",Политика
4,Пресс-служба президента предсказала серьёзные ...,На брифинге в пятницу пресс-секретарь президен...,Политика


In [ ]:
dataset = pd.concat([anna, gleb])
dataset.shape

(9097, 3)

In [ ]:
dataset = dataset.sample(frac = 1)
# dataset.head()

##Предобработка данных

###Очистка данных

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('russian'))

In [ ]:
import pandas as pd
data = pd.read_csv('/content/dataset.csv')
data

In [ ]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data

In [ ]:
headings=list(data)

In [ ]:
def f(data):
  headings=list(data)
  import nltk
  nltk.download('stopwords')
  from nltk.corpus import stopwords
  print(stopwords.words('russian'))
  from nltk.corpus import stopwords
  from nltk.tokenize import word_tokenize
  nltk.download('punkt')
  stop_words = set(stopwords.words('russian'))
  print(type(stop_words))
  import re
  for i in range(0, len(headings)):
    for j in range(len(data)):
      if type(data[headings[i]][j])==str:
        data[headings[i]][j] = re.sub(" +", " ", data[headings[i]][j])
        data[headings[i]][j] = data[headings[i]][j].lower()
        data[headings[i]][j] = re.sub("[^\\dа-я !.,?:;]","",data[headings[i]][j])
        data[headings[i]][j] = re.sub(" - ","",data[headings[i]][j])
        word_tokens = word_tokenize(data[headings[i]][j])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        data[headings[i]][j] = filtered_sentence

In [ ]:
f(data)

###Леммитизация

In [ ]:
def lemmitization(words):
  morph = pymorphy2.MorphAnalyzer()
  words_lem = []
  # print('Still working...')
  for word in words:
    words_lem.append(morph.parse(word)[0].normal_form)

  return ' '.join(words_lem)

for i in range(data.shape[0]):
    data.loc[i, 'Text'] = lemmitization(data.loc[i, 'Text'].split(' '))

In [ ]:
!wget https://www.dropbox.com/s/mjybhw3t67wk6zi/data_lemm.csv?dl=0 -O data_lemm.csv

--2021-12-15 22:41:10--  https://www.dropbox.com/s/mjybhw3t67wk6zi/data_lemm.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/mjybhw3t67wk6zi/data_lemm.csv [following]
--2021-12-15 22:41:11--  https://www.dropbox.com/s/raw/mjybhw3t67wk6zi/data_lemm.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0d7b0c9be0564f1c3f71caab3d.dl.dropboxusercontent.com/cd/0/inline/Bb58QPAbBVDDAsNlF2n8HrMvk-DBZ5I8_L5GG30jKHIvDMrZA2WGHf3tQcVH-IY4XKzDeqjlui-Brcy63AI0HN4M996nxoF0OOu0wIozhqS5r6gAFoAvSUPq3f_joY5mCAf0tHDFJqLfG3BuhXuY6GXw/file# [following]
--2021-12-15 22:41:11--  https://uc0d7b0c9be0564f1c3f71caab3d.dl.dropboxusercontent.com/cd/0/inline/Bb58QPAbBVDDAsNlF2n8HrMvk-DBZ5I8_L5GG30jKHIvDMrZA2WGHf3tQcVH-IY4XKzDeqjlui-Brcy

In [ ]:
data = pd.read_csv('/content/data_lemm.csv')
# data.dropna(inplace=True)
# data.reset_index(drop=True, inplace=True)
# data.replace('\d+', '', regex=True, inplace=True)
# data.replace("^a-zA-Z0-9]+", "", regex=True, inplace=True)
data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Header,Text,Category,class
0,0,0,4175,"«Я рада, что помогла вам обрести надежду и под...",приятный сюрприз новый год зритель решить сдел...,Общество,0
1,1,1,2138,«Первый канал» проведёт телемарафон для спасен...,первый канал декабрь провести благотворительны...,Общество,0
2,2,2,161,Навального перевели в ШИЗО на дня за татуиров...,блогер алексей навальный отбывать срок исправи...,Политика,1
3,3,3,8525,По итогам года Беларусь импортировала из РФ тр...,итог месяц год республика беларусь импортиров...,Экономика,2
4,4,4,4062,В Сколково разработали умный автоответчик для ...,российский программист совместно инженер иннов...,Общество,0
...,...,...,...,...,...,...,...
9090,9090,9092,8485,"В Tesla появится автопилот, способный ездить п...",автоконцерн tesla планировать выпуск новый мод...,Экономика,2
9091,9091,9093,8264,Во французской палате мер и весов появится эта...,международный бюро вес франция сообщить появле...,Наука,3
9092,9092,9094,4596,ЦРУ: «Навальный не работает на нас. Он уволилс...,представитель центральный разведывательный упр...,Общество,0
9093,9093,9095,5910,"Москвичи подожгли мусорный полигон, дождавшись...",московский область февраль группа экстремистск...,Общество,0


In [ ]:
# data['class'] = 0
# data.loc[data['Category'] == 'Политика', 'class'] = 1
# data.loc[data['Category'] == 'Экономика', 'class'] = 2
# data.loc[data['Category'] == 'Наука', 'class'] = 3
# data.loc[data['Category'] == 'Статьи', 'class'] = 4
# data.loc[data['Category'] == 'Книги', 'class'] = 5



In [ ]:
data.to_csv('./data_lemm.csv')

## Векторизация

###TF/IDF

In [ ]:
def tfidf(texts):

    corpus = texts.to_numpy()
    
    vocabulary = list(set([item for sublist in list(data['Text'].astype(str)) for item in sublist.split()]))
    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                    ('tfid', TfidfTransformer())]).fit(corpus)
    pipe['count'].transform(corpus).toarray()
    pipe['tfid'].idf_

    return pipe.transform(corpus)

test_tfidf = tfidf(data['Text'].astype(str))

###Bag of Words:

In [ ]:
def corpuses_embedding_count_vectorize(texts):

    corpus = texts.to_numpy()
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)

    return X.toarray()

test_bow = corpuses_embedding_count_vectorize(data['Text'].astype(str))

###Doc2Vec:

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def doc_to_vec(texts):
    #tokenize and tag the card text
    card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(data['Text'].astype(str))]

    model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
    #build vocab
    model.build_vocab(card_docs)
    #train model
    model.train(card_docs, total_examples=model.corpus_count
                , epochs=model.epochs)

    card2vec = [model.infer_vector((texts[i].split(' '))) 
            for i in range(0,len(texts))]
    
    dtv= np.array(card2vec).tolist()

    return dtv

test_doc2vec = doc_to_vec(data['Text'])


Fast Text не надо

In [ ]:
from gensim.models.fasttext import FastText

def ft(texts):


    embedding_size = 60
    window_size = 40
    min_word = 5
    down_sampling = 1e-2

    ft_model = FastText(texts,
                        size=embedding_size,
                        window=window_size,
                        min_count=min_word,
                        sample=down_sampling,
                        sg=1,
                        iter=1)
    
    return np.array([[ft_model.wv[word] for word in text] for text in data['Text']])


## Обучаем классификатор
Смотрим по метрике F1-score

Импорты:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

Функция обучения:

In [ ]:
def train(model, X, y): 
  # тут разбиваем на train/test
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # учим
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  # и можно еще принтануть F1_score внутри
  print("F1 score = ", f1_score(y_test, y_pred, average='weighted'))

  # возвращаем y_test y_pred
  return [y_test, y_pred, f1_score(y_test, y_pred, average='weighted')]

### Байесовский классификатор

In [ ]:
gnb = GaussianNB()

Doc2Vec:

In [ ]:
d2v = train(gnb, test_doc2vec, data['class']) 

F1 score =  0.7057905914961834


TF/IDF:

In [ ]:
tfidf = train(gnb, test_tfidf.toarray(), data['class'])

Bag of words:

In [ ]:
bow = train(gnb, test_bow, data['class'])

In [ ]:
df = pd.DataFrame(np.array([['d2v', 'tfidf', 'bow'], ['naive_bayes', 'naive_bayes', 'naive_bayes'], [d2v[2], tfidf[2], bow[2]]]),
                   columns=['preprocess', 'model', 'f1 score'])
df

NameError: ignored

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

Doc2Vec

In [ ]:
d2v_rfc = train(rfc, test_doc2vec, data['class'])

TF/IDF:

In [ ]:
tfidf_rfc = train(rfc, test_tfidf.toarray(), data['class'])

Bag of words:

In [ ]:
bow_rfc = train(rfc, test_bow, data['class'])

In [ ]:
df_rfc = pd.DataFrame(np.array([['d2v_rfc', 'tfidf_rfc', 'bow_rfc'], ['random_forest', 'random_forest', 'random_forest'], [d2v_rfc[2], tfidf_rfc[2], bow_rfc[2]]]),
                   columns=['preprocess', 'model', 'f1 score'])
df_rfc

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

Doc2Vec

In [ ]:
d2v_knn = train(knn, test_doc2vec, data['class'])

TF/IDF:

In [ ]:
tfidf_knn = train(knn, test_tfidf.toarray(), data['class'])

Bag of words:

In [ ]:
bow_knn = train(knn, test_bow, data['class'])

In [ ]:
df_knn = pd.DataFrame(np.array([['d2v_knn', 'tfidf_knn', 'bow_knn'], ['knn', 'knn', 'knn'], [d2v_knn[2], tfidf_knn[2], bow_knn[2]]]),
                   columns=['preprocess', 'model', 'f1 score'])
df_knn

### SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()

Doc2Vec

In [ ]:
d2v_svc = train(svc, test_doc2vec, data['class'])

TF/IDF:

In [ ]:
tfidf_svc = train(svc, test_tfidf.toarray(), data['class'])

Bag of words:

In [ ]:
bow_svc = train(svc, test_svc, data['class'])

In [ ]:
df_svc = pd.DataFrame(np.array([['d2v_svc', 'tfidf_svc', 'bow_svc'], ['svc', 'svc', 'svc'], [d2v_svc[2], tfidf_svc[2], bow_svc[2]]]),
                   columns=['preprocess', 'model', 'f1 score'])
df_svc